In [1]:
import os
import io
import sys
import psutil
import random
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchinfo
from torchinfo import summary
from torch.utils.checkpoint import checkpoint
import torch.profiler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import euclidean

# Add the parent directory, i.e. transformer, means parent directory of 'scripts' and 'notebooks', to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Import classes and functions
from scripts.m1_functions import *
from scripts.m1_classes import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
def data_loader_filtered_imu():
    '''
    Read in csv file of the entire dataset. If not already created, read-in all individual csv files for all subjects and actions. 
    Add subject and action column for labeling the actions. Filter the signals with an appropriate bandpass filter.
    '''
    if os.path.exists('../data/Finger/csv/finger_dataset_filtered_imu.csv'):
        # Reading the CSV file
        df_filtered = pd.read_csv(
                    '../data/Finger/csv/finger_dataset_filtered_imu.csv',
                    sep=',',           
                    header=0,          
                    na_values=['NA', '']  
        )
        return df_filtered
    
    else:                    
        actions = ["run", "sit", "walk"]
        subjects = [i for i in range(1 , 23)]
        df_data = pd.DataFrame()
        for subject in subjects:
            for action in actions:
                file_path = f'../data/Finger/csv/s{str(subject)}_{action}.csv'
                # Reading the CSV file
                try:
                    df_temp = pd.read_csv(
                        file_path,
                        sep=',',           
                        header=0,          
                        na_values=['NA', '']  
                    )
                    # Adding a column with the current action and subject
                    df_temp['action'] = action
                    df_temp['subject'] = subject
                    df_data = pd.concat([df_data, df_temp], ignore_index=True)

                except FileNotFoundError:
                    print(f"File not found: {file_path}")
        # Define sampling frequency, bandpass range and apply bandpass filter
        df_filtered = pd.DataFrame()
        fs = 500  
        lowcut = 0.4
        highcut = 10
        df_filtered['ecg']  = bandpass_filter(df_data['ecg'], lowcut, highcut, fs, order=4)
        df_filtered['red ppg'] = bandpass_filter(df_data['pleth_4'], lowcut, highcut, fs, order=4)
        df_filtered['ir ppg'] = bandpass_filter(df_data['pleth_5'], lowcut, highcut, fs, order=4)
        df_filtered['green ppg'] = bandpass_filter(df_data['pleth_6'], lowcut, highcut, fs, order=4)
        df_filtered['action'] = df_data['action']
        df_filtered['subject'] = df_data['subject']
        df_filtered['a_x'] = df_data['a_x']
        df_filtered['a_y'] = df_data['a_y']
        df_filtered['a_z'] = df_data['a_z']
        df_filtered['g_x'] = df_data['g_x']
        df_filtered['g_y'] = df_data['g_y']
        df_filtered['g_z'] = df_data['g_z']
        # Store the DataFrame to a csv file
        df_filtered.to_csv('../data/Finger/csv/finger_dataset_filtered_imu.csv', index=False)
        return df_filtered

In [3]:
device = select_device()

Using mps device


In [4]:
df_filtered = data_loader_filtered_imu()

In [3]:
df_filtered = data_loader_filtered_imu()
df_original = data_loader_original()
df_filtered_single = data_loader_filtered_single(subject=10, action='sit')
df_original_single = data_loader_original_single(subject=10, action='sit')

In [5]:
def normalization_group_action_imu(df):
    '''
    Group the data by subject and actions, and normalize each (subject, action) pair
    '''
    # Initialize a dictionary to store scalers for each subject-action group
    scalers = {}

    # Placeholder for normalized data
    normalized_data = []

    # Group data by subject and action
    grouped_data = df.groupby(['subject', 'action'])
    print(grouped_data.first())

    for (subject, action), group in grouped_data:
        # Initialize scalers for PPG (inputs) and ECG (targets)
        scaler_input = MinMaxScaler(feature_range=(-1, 1))  # For PPG signals
        scaler_target = MinMaxScaler(feature_range=(-1, 1))  # For ECG signals

        # Fit and transform the PPG and IMU columns (inputs)
        ppg_normalized = scaler_input.fit_transform(group[['red ppg', 'ir ppg', 'green ppg', 'a_x', 'a_y', 'a_z', 'g_x', 'g_y', 'g_z']])

        # Fit and transform the ECG column (target)
        ecg_normalized = scaler_target.fit_transform(group[['ecg']])

        # Save the scalers for this subject-action group
        scalers[(subject, action)] = {'input_scaler': scaler_input, 'target_scaler': scaler_target}
        # Inspect original scalers (collective normalization)
        print(scalers[(subject, action)]['input_scaler'].data_min_, scalers[(subject, action)]['input_scaler'].data_max_)

        # Create a copy of the group with normalized values
        group_normalized = group.copy()
        group_normalized[['red ppg', 'ir ppg', 'green ppg', 'a_x', 'a_y', 'a_z', 'g_x', 'g_y', 'g_z']] = ppg_normalized
        group_normalized[['ecg']] = ecg_normalized

        # Append the normalized group to the list
        normalized_data.append(group_normalized)

    # Combine all normalized groups back into a single DataFrame
    normalized_df = pd.concat(normalized_data).reset_index(drop=True)
    return normalized_df, scalers

In [6]:
df_custom_normalized, scalers = normalization_group_action_imu(df_filtered)


                        ecg     red ppg      ir ppg  green ppg        a_x  \
subject action                                                              
1       run      174.974839   15.310121   36.949744   2.448764   1.264203   
        sit    -2385.170850  -60.995543 -207.180828 -27.146285   4.298409   
        walk    3103.829398  -72.695792 -206.359678 -41.808654   9.262442   
2       run    -4260.135210   62.017004  212.866797 -69.501723   1.019383   
        sit     -512.615543   97.781635  205.756513  32.764378   8.602206   
...                     ...         ...         ...        ...        ...   
21      sit    -3314.057062   76.640336  125.034479  10.588793   1.431806   
        walk   -1528.150596 -104.075541  -65.526139 -27.216916  11.171316   
22      run    -2490.481940  -11.945232  147.605710  48.883196   4.551610   
        sit     4405.402837  -30.184581  -41.718308   0.660463   1.579057   
        walk     589.877914 -106.935586 -130.246244 -37.167680   9.995703   

In [ ]:
# Define the subjects and action you want to filter
selected_subjects = [1,2,3,4,5]  # Replace with desired subject IDs
selected_action = 'sit'    # Replace with the desired action

# Filter the DataFrame
df_custom = df_filtered[(df_filtered['subject'].isin(selected_subjects)) & (df_filtered['action'] == selected_action)]

# Reset the index of the new DataFrame
df_custom = df_custom.reset_index(drop=True)

# Verify the result
print(df_custom.head())
print(f"New DataFrame shape: {df_custom.shape}")
df_custom

In [6]:
start = 253000
stop = 255000

In [ ]:
# Define the time points
time = df_custom.index  # Assuming your dataframe has a time-based index

# Plot the normalized ECG time-series
plt.figure(figsize=(10, 6))
plt.plot(time[start:stop], df_custom['ecg'][start:stop], label="Normalized ECG Signal", color='blue')
plt.title("Normalized ECG Time-Series")
plt.xlabel("Time")
plt.ylabel("Normalized Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

# Plot all PPG signals and ECG on the same graph
plt.subplot(2, 1, 1)
plt.plot(time[start:stop], df_custom['red ppg'][start:stop], label=" Red PPG", color='red')
plt.plot(time[start:stop], df_custom['green ppg'][start:stop], label=" Green PPG", color='green')
plt.plot(time[start:stop], df_custom['ir ppg'][start:stop], label=" IR PPG", color='purple')
plt.title(" PPG Signals")
plt.xlabel("Time")
plt.ylabel(" Value")
plt.legend()

# Plot ECG signal
plt.subplot(2, 1, 2)
plt.plot(time[start:stop], df_custom['ecg'][start:stop], label=" ECG", color='blue')
plt.title(" ECG Signal")
plt.xlabel("Time")
plt.ylabel(" Value")
plt.legend()

plt.tight_layout()
plt.show()

In [9]:
# df_custom_normalized, scalers = normalization_group_action(df_custom)
# df_custom_normalized

In [ ]:
df_custom_normalized, scalers = global_normalization(df_custom)
df_custom_normalized

In [ ]:
plt.figure(figsize=(12, 8))

# Plot all PPG signals and ECG on the same graph
plt.subplot(2, 1, 1)
plt.plot(time[start:stop], df_custom_normalized['red ppg'][start:stop], label="Normalized Red PPG", color='red')
plt.plot(time[start:stop], df_custom_normalized['green ppg'][start:stop], label="Normalized Green PPG", color='green')
plt.plot(time[start:stop], df_custom_normalized['ir ppg'][start:stop], label="Normalized IR PPG", color='purple')
plt.title("Normalized PPG Signals")
plt.xlabel("Time")
plt.ylabel("Normalized Value")
plt.legend()

# Plot ECG signal
plt.subplot(2, 1, 2)
plt.plot(time[start:stop], df_custom_normalized['ecg'][start:stop], label="Normalized ECG", color='blue')
plt.title("Normalized ECG Signal")
plt.xlabel("Time")
plt.ylabel("Normalized Value")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Define the time points
time = df_custom_normalized.index  # Assuming your dataframe has a time-based index

# Plot the normalized ECG time-series
plt.figure(figsize=(10, 6))
plt.plot(time[253000:255000], df_custom_normalized['ecg'][253000:255000], label="Normalized ECG Signal", color='blue')
plt.title("Normalized ECG Time-Series")
plt.xlabel("Time")
plt.ylabel("Normalized Value")
plt.legend()
plt.grid(True)
plt.show()

In [7]:
def sequences_imu(df, sequence_length, sequence_step_size, subset):
    '''
    Create sequnces with the per (subject, action) pair normalized dataframe
    '''
    # Retrieve input ppg signals
    input_columns = ['red ppg', 'ir ppg', 'green ppg', 'a_x', 'a_y', 'a_z', 'g_x', 'g_y', 'g_z']
    x_normalized = df[input_columns].values

    # Retrieve target ecg signals
    y_normalized = df[['ecg']].values

    # Convert to PyTorch tensors
    x_data = torch.tensor(x_normalized, dtype=torch.float32)  # Shape: [samples, 6]
    y_data = torch.tensor(y_normalized, dtype=torch.float32)  # Shape: [samples, 1]

    # Reshape for sequence length and adjustable stepsize. Sequences are shifted by timestamp / sample stepsize per sequence! 
    num_sequences = len(df) - sequence_length + 1

    x_sequences = torch.stack([x_data[i:i + sequence_length] for i in range(0, int(num_sequences*subset), int(sequence_step_size))])  # [num_sequences, seq_length, 6]
    y_sequences = torch.stack([y_data[i:i + sequence_length] for i in range(0, int(num_sequences*subset), int(sequence_step_size))])  # [num_sequences, seq_length, 1]
    return x_sequences, y_sequences

In [8]:
df_custom_normalized

,ecg,red ppg,ir ppg,green ppg,action,subject,a_x,a_y,a_z,g_x,g_y,g_z
0,-0.164368,-0.735236,-0.739668,-0.745664,run,1,-0.204395,-0.343275,-0.155174,-0.399232,0.188411,0.079801
1,-0.158655,-0.734529,-0.738882,-0.746721,run,1,-0.238707,-0.357213,-0.154050,-0.396621,0.205572,0.089421
2,-0.152808,-0.733826,-0.738097,-0.747785,run,1,-0.261582,-0.342525,-0.165288,-0.391091,0.218713,0.096904
3,-0.146828,-0.733132,-0.737317,-0.748848,run,1,-0.278031,-0.317048,-0.185923,-0.385254,0.225827,0.101045
4,-0.140714,-0.732452,-0.736544,-0.749902,run,1,-0.297565,-0.293818,-0.211666,-0.380107,0.227348,0.101780
...,...,...,...,...,...,...,...,...,...,...,...,...
16217127,0.621431,0.735613,0.697284,0.705999,walk,22,-0.455802,0.051600,-0.370925,-0.396948,-0.124408,0.033390
16217128,0.563686,0.736206,0.698010,0.706974,walk,22,-0.445030,0.058361,-0.354496,-0.388573,-0.142180,0.029919
16217129,0.504797,0.736785,0.698713,0.707919,walk,22,-0.438941,0.064869,-0.336357,-0.380008,-0.160308,0.025741
16217130,0.445473,0.737347,0.699388,0.708826,walk,22,-0.434258,0.070363,-0.314965,-0.371189,-0.177607,0.021178


In [9]:
# Ratios for train, validation, and test splits
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

sequence_length = 1000
sequence_step_size = 100
subset = 0.001

# Generate sequences
x_data, y_data = sequences_imu(df_custom_normalized, sequence_length, sequence_step_size, subset)

# Calculate sizes for each subset
total_samples = x_data.size(0)
train_size = int(train_ratio * total_samples)
val_size = int(val_ratio * total_samples)
test_size = total_samples - train_size - val_size  # Remaining samples go to the test set

# Split the data
X_train = x_data[:train_size]
y_train = y_data[:train_size]

X_val = x_data[train_size:train_size + val_size]
y_val = y_data[train_size:train_size + val_size]

X_test = x_data[train_size + val_size:]
y_test = y_data[train_size + val_size:]


# Print shapes for verification
print(f"x_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"x_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

x_train shape: torch.Size([114, 1000, 9]), y_train shape: torch.Size([114, 1000, 1])
x_val shape: torch.Size([32, 1000, 9]), y_val shape: torch.Size([32, 1000, 1])
x_test shape: torch.Size([17, 1000, 9]), y_test shape: torch.Size([17, 1000, 1])


In [ ]:
X_train.shape

In [16]:
# Model initialization 
d_model = 64  # Embedding dimension
input_dim = 9  # 3 PPG signals (red, green, IR)
output_dim = 1  # 1 ECG target per time step
nhead = 2  # Attention heads
num_layers = 2  # Number of transformer layers
batch_size = 8  # Batch size

seed = 42

# Convert tensors to Datasets
train_dataset = PreprocessedDataset(X_train, y_train)
val_dataset = PreprocessedDataset(X_val, y_val)

# Create DataLoaders with a reproducible generator
gen = torch.Generator(device=device)
gen.manual_seed(seed)

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, generator=gen)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Initialize the Transformer model
model = Point2PointEncoderTransformer(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers).to(device) 

X_train_sample = X_train[:1]
y_train_sample = y_train[:1]

# Call the torchinfo summary method
#summary_txt = summary(model, input_data=X_train_sample, depth=1, device=device)
#print(summary_txt)

In [14]:
c = 0
for batch_X, batch_y in train_loader:
    batch_X = batch_X.to(device)
    batch_y = batch_y.to(device)
    c +=1
    
print(batch_X.shape)
print(c)

torch.Size([2, 1000, 9])
15


In [ ]:
batch_X.shape

In [17]:
# Loss function: Mean Squared Error for regression tasks
loss_fn = nn.MSELoss()

# Optimizer: Adam optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001,betas=(0.9,0.999),eps=1e-08,weight_decay=0.01,amsgrad=False)

# Number of epochs to train
num_epochs = 10

# Initialize a learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6)


# Clear any residual memory before training
torch.cuda.empty_cache()

# Arrays for storing losses and epochs
training_loss = np.array([])
validation_loss = np.array([])
epochs = np.array([])
best_models = np.array([])

# Early stopping and checkpoint parameters
patience = 10
min_delta = 1e-4
best_val_loss = float('inf')
early_stop_counter = 0


### Training
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0 # Initialize running loss
    # Iterate through the batches in the train_loader to load the data in batches
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass through the model
        predictions = model(batch_X)

        # Calculate loss (MSE between predicted ECG and actual ECG)
        loss = loss_fn(predictions, batch_y)

        # Backward pass (compute gradients)
        loss.backward()

        # Update the weights
        optimizer.step()

        # Update running loss
        running_loss += loss.item() * batch_X.size(0)
    

    # Calculate the average loss for the epoch
    avg_train_loss = running_loss / len(X_train)
    train_rmse = torch.sqrt(torch.tensor(avg_train_loss)) # MSE needs to be calculated at the end of each batch, scaled by batch size and the RMSE should calculated at the end of the epoch (metric)
    training_loss = np.append(training_loss, train_rmse.cpu())

    print(f"Training of epoch {epoch+1} done, starting validation!")
    # Validation metrics with batching
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0

    with torch.no_grad():
        # Iterate through the batches in the val_loader to load the data in batches
        for batch_X_val, batch_y_val in val_loader:
            batch_X_val = batch_X_val.to(device)
            batch_y_val = batch_y_val.to(device)
        
            # Forward pass
            val_predictions = model(batch_X_val)

            # Calculate loss for this batch
            val_loss = loss_fn(val_predictions, batch_y_val)

            # Accumulate total validation loss
            total_val_loss += val_loss.item() * batch_X_val.size(0)  # Weighted by batch size


        # Average validation loss over all samples
        avg_val_loss = total_val_loss / len(X_val) 
        val_rmse = torch.sqrt(torch.tensor(avg_val_loss)) # MSE needs to be calculated at the end of each batch, scaled by batch size and the RMSE should calculated at the end of the epoch (metric)
        validation_loss = np.append(validation_loss, val_rmse.cpu())

        # Step the learning rate scheduler with the validation loss
        scheduler.step(avg_val_loss)

        # Early stopping
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            early_stop_counter = 0
            # Save checkpoint
            #checkpoint_path = f"{checkpoints_folder}/epoch{epoch+1}.pth"
            #save_checkpoint(model, optimizer, epoch + 1, avg_val_loss, checkpoint_path)

            # Save the model
            #torch.save(model.state_dict(), f"../models/{model_family}{model_name}_trained_model_epoch{epoch+1}.pth")

            # Save the epoch of this best model
            best_models = np.append(best_models, int(epoch+1))

            epochs = np.append(epochs, int(epoch+1))
            print(f"Checkpoint saved at epoch {epoch + 1}.")
        else:
            epochs = np.append(epochs, int(epoch+1))
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break

        #print(f"Memory usage: {psutil.virtual_memory().percent}%")
        print(f"Epoch {epoch + 1}/{num_epochs} | Train RMSE: {train_rmse:.4f} | Val RMSE: {val_rmse:.4f} | Current LR: {optimizer.param_groups[0]['lr']:.6f}")

        # Clear any residual memory before start of new epoch
        torch.cuda.empty_cache()

Training of epoch 1 done, starting validation!
Checkpoint saved at epoch 1.
Epoch 1/10 | Train RMSE: 0.2946 | Val RMSE: 0.2003 | Current LR: 0.001000
Training of epoch 2 done, starting validation!
Checkpoint saved at epoch 2.
Epoch 2/10 | Train RMSE: 0.2192 | Val RMSE: 0.1698 | Current LR: 0.001000
Training of epoch 3 done, starting validation!
Epoch 3/10 | Train RMSE: 0.2817 | Val RMSE: 0.2661 | Current LR: 0.001000
Training of epoch 4 done, starting validation!
Epoch 4/10 | Train RMSE: 0.1930 | Val RMSE: 0.2553 | Current LR: 0.001000
Training of epoch 5 done, starting validation!
Epoch 5/10 | Train RMSE: 0.1762 | Val RMSE: 0.2369 | Current LR: 0.001000
Training of epoch 6 done, starting validation!
Epoch 6/10 | Train RMSE: 0.1726 | Val RMSE: 0.2387 | Current LR: 0.001000
Training of epoch 7 done, starting validation!
Epoch 7/10 | Train RMSE: 0.1694 | Val RMSE: 0.2268 | Current LR: 0.001000
Training of epoch 8 done, starting validation!
Epoch 8/10 | Train RMSE: 0.1670 | Val RMSE: 0.22

In [20]:
df_test = df_filtered

In [ ]:
df_custom

In [ ]:
df_test

In [ ]:
df_filtered

In [21]:
scalers

{(1, 'run'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (1, 'sit'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (1, 'walk'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (2, 'run'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (2, 'sit'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (2, 'walk'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (3, 'run'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (3, 'sit'): {'input_scaler': MinMaxScaler(feature_range=(-1, 1)),
  'target_scaler': MinMaxScaler(feature_range=(-1, 1))},
 (3, '

In [22]:
### Validation
# Initialize storage for aggregated predictions and actual values
ecg_predictions = []
ecg_actuals = []
ppg = []
subjects = []  # Store subject info for each batch
actions = []   # Store action info for each batch

# Loss function: Mean Squared Error for regression tasks
loss_fn = nn.MSELoss()

# Test Loss
test_loss = np.array([])
running_test_loss = 0

# Convert tensors to Datasets
test_dataset = PreprocessedDataset(X_test, y_test)
# Create DataLoaders for each set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Iterate over the validation set in batches
model.eval()  # Ensure the model is in evaluation mode
with torch.no_grad():
    # Iterate through the batches in the test_loader to load the data in batches
    for batch_idx, (batch_X_test, batch_y_test) in enumerate(test_loader):
        # Move the batch data to the device (GPU or CPU)
        batch_X_test = batch_X_test.to(device)
        batch_y_test = batch_y_test.to(device)
        
        # Get the start and end index of the current batch in df_test
        start_idx = batch_idx * batch_size
        end_idx = start_idx + len(batch_X_test)
        
        # Retrieve the corresponding (subject, action) pair for this batch from df_test
        batch_subjects = df_test.iloc[start_idx:end_idx]['subject'].values
        batch_actions = df_test.iloc[start_idx:end_idx]['action'].values

        # Forward pass to get predictions
        batch_predictions = model(batch_X_test)

        # Calculate loss for this batch
        loss = loss_fn(batch_predictions, batch_y_test)

        # Accumulate total validation loss
        running_test_loss += loss.item() * batch_X_test.size(0)

        # Store predictions, actuals, subjects, and actions
        ecg_predictions.append(batch_predictions.cpu())  # Move to CPU for numpy/scaler operations
        ecg_actuals.append(batch_y_test.cpu())
        ppg.append(batch_X_test.cpu())
        subjects.extend(batch_subjects)
        actions.extend(batch_actions)

# Average the test loss over all samples
avg_test_loss = running_test_loss / len(X_test)
test_rmse = torch.sqrt(torch.tensor(avg_test_loss))
test_loss = np.append(test_loss, test_rmse.cpu())

# Concatenate all batches
ecg_predictions = torch.cat(ecg_predictions, dim=0)
ecg_actuals = torch.cat(ecg_actuals, dim=0)
ppg = torch.cat(ppg, dim=0)

# Initialize lists for original scale data
ecg_predictions_original_scale = []
ecg_actuals_original_scale = []
ppg_original_scale = []

# Process each sequence
for i in range(len(ecg_predictions)):
    # Get subject and action for the current sequence
    subject = subjects[i]
    action = actions[i]

    # Retrieve the correct scalers
    scaler_input = scalers['input_scaler']
    scaler_target = scalers['target_scaler']

    # Inverse transform predictions and actuals for the current sequence
    ecg_pred = ecg_predictions[i].squeeze(-1).numpy()  # Shape: [sequence_length]
    ecg_act = ecg_actuals[i].squeeze(-1).numpy()       # Shape: [sequence_length]
    ppg_seq = ppg[i].numpy()                          # Shape: [sequence_length, 3]

    ecg_predictions_original_scale.append(scaler_target.inverse_transform(ecg_pred.reshape(-1, 1)).flatten())
    ecg_actuals_original_scale.append(scaler_target.inverse_transform(ecg_act.reshape(-1, 1)).flatten())
    ppg_original_scale.append(scaler_input.inverse_transform(ppg_seq))

# Convert back to arrays
ecg_predictions_original_scale = np.array(ecg_predictions_original_scale)
ecg_actuals_original_scale = np.array(ecg_actuals_original_scale)
ppg_original_scale = np.array(ppg_original_scale)

# Separate PPG channels 
red_ppg = ppg_original_scale[:, :, 0]  # Red PPG
ir_ppg = ppg_original_scale[:, :, 1]   # IR PPG
green_ppg = ppg_original_scale[:, :, 2]  # Green PPG


### Normalized Evaluation metrics
# Predictions and actual values (normalized and flattened)
ecg_predictions_arr = np.array(ecg_predictions).flatten()
ecg_actuals_arr = np.array(ecg_actuals).flatten()

# Calculate the range of the actual data for normalization
actual_range_normalized = np.ptp(ecg_actuals)  # Peak-to-peak (max - min)

# Euclidean Distance
euclidean_distance_normalized = euclidean(ecg_predictions_arr, ecg_actuals_arr)

# Dynamic Time Warping (DTW)
downsampling_factor_dtw = 10
batch_size_dtw = 10 
dtw_distance_normalized = compute_batched_dtw(ecg_predictions_arr, ecg_actuals_arr, batch_size_dtw, downsampling_factor_dtw)
# dtw_distance = alignment.distance

# Pearson Correlation
pearson_corr_normalized, _ = pearsonr(ecg_predictions_arr, ecg_actuals_arr)

# Spearman Correlation
spearman_corr_normalized, _ = spearmanr(ecg_predictions_arr, ecg_actuals_arr)

# Mean Squared Error (MSE)
mse_normalized = np.mean((ecg_predictions_arr - ecg_actuals_arr) ** 2)

# Mean Absolute Error (MAE)
mae_normalized = np.mean(np.abs(ecg_predictions_arr - ecg_actuals_arr))

# Root Mean Squared Error (RMSE)
rmse_normalized = np.sqrt(mse_normalized)

# Normalized Root Mean Squared Error (NRMSE)
nrmse_normalized = rmse_normalized / actual_range_normalized

# Normalized Mean Absolute Error (NMAE)
nmae_normalized = mae_normalized / actual_range_normalized

# Print metrics
metrics_normalized = {
    "Training_loss": training_loss,
    "Validation_loss": validation_loss,
    "Test_loss": test_loss,
    "Epochs": epochs, 
    "Euclidean Distance": euclidean_distance_normalized,
    "DTW Distance": dtw_distance_normalized,
    "Pearson Correlation": pearson_corr_normalized,
    "Spearman Correlation": spearman_corr_normalized,
    "MSE": mse_normalized,
    "MAE": mae_normalized,
    "RMSE": rmse_normalized,
    "NRMSE": nrmse_normalized,
    "NMAE": nmae_normalized,
    # "Parameters": config['parameters'],  # Add config file entries
    # "General": config['general'],
    # "Output": config['output'],
}

for metric, value in metrics_normalized.items():
    #print(f"{metric}: {value:.4f}")
    print(f"{metric}: {value}")

KeyError: 'input_scaler'

In [ ]:
ppg_original_scale

In [ ]:
ecg_predictions_original_scale


In [ ]:
### Plots
# Call a plot(...) method to create them
# Randomly select an index from the validation data
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss,  label='Training Loss')
plt.plot(epochs, validation_loss, label='Validation Loss')
plt.yscale('log')
plt.title(f"Training and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
#plt.xticks(epochs)
plt.legend()

#plt.savefig(f"{results_folder}/{model_name}_loss_functions.png")

    # Repeat test loss across all epochs for visualization
test_losses = [test_loss] * len(epochs)

# Plot the losses
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, label="Training Loss", marker='o')
plt.plot(epochs, validation_loss, label="Validation Loss", marker='o')
plt.plot(epochs, test_losses, label="Test Loss", linestyle='--', color='red')

# Add labels, title, legend
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training, Validation, and Test Loss")
plt.legend()

#plt.savefig(f"{results_folder}/{model_name}_test_loss.png")

#random_index = np.random.randint(0, len(ecg_predictions_original_scale))
random_index = 1
ppg_scaling_factor = 100

# Select the corresponding actual and predicted ECG signals
ecg_predictions_random = ecg_predictions_original_scale[random_index]  # Predicted ECG signal
ecg_actuals_random = ecg_actuals_original_scale[random_index]  # Actual ECG signal

# Set the opacity value of alpha for the ppg signals
alpha = 0.3

# Plot the actual and predicted ECG
plt.figure(figsize=(10, 5))
plt.plot(ecg_actuals_random, label='Actual ECG')
plt.plot(ecg_predictions_random, label='Predicted ECG')
plt.title(f"ECG Prediction vs Actual (Sequence {random_index})")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

#plt.savefig(f"{results_folder}/{model_name}_random_seq.png")

# Plot the actual and predicted ECG with the input ppg signals
plt.figure(figsize=(10, 5))
plt.plot(ecg_actuals_random, label='Actual ECG')
plt.plot(ecg_predictions_random, label='Predicted ECG')
plt.plot(ppg_scaling_factor*red_ppg[random_index], label="Red PPG", alpha=alpha)
plt.plot(ppg_scaling_factor*ir_ppg[random_index], label="IR PPG", alpha=alpha)
plt.plot(ppg_scaling_factor*green_ppg[random_index], label="Green PPG", alpha=alpha)
plt.title(f"ECG Prediction vs Actual (Sequence {random_index}) with PPG signals")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

#plt.savefig(f"{results_folder}/{model_name}_random_seq_ppg.png")

print("Evaluation finished!")